In [65]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D, Activation
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

import numpy as np
import PIL
from PIL import Image
import json
import time
import requests
# from StringIO import StringIO     # python2 version
from io import BytesIO     # python3 version
import os.path
import sys
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [24]:
def loadImageUrl(imgUrl):
    response = requests.get(imgUrl)
    return Image.open(BytesIO(response.content))
def loadImageFile(imgFile):
    return Image.open(imgFile)

def loadImage(imageFile,size=224):
    if imageFile.startswith('http'):
        img = loadImageUrl(imageFile)
    elif os.path.isfile(imageFile):
        img = loadImageFile(imageFile)
    else:
        return {"file": imageFile, "message": "Not Found"}
    img = img.resize((size, size), PIL.Image.ANTIALIAS)
    img = img.convert('RGB')
    return img


In [29]:
def data_prepared():
# gether all trained and test data
# in test period-1, we add data from local, used pointer to point the image in folder
# in test period-2, we add data from website,

    url1='./data/1/'
    url2='./data/2/'
    class1_dir = [url1+i for i in os.listdir(url1)]
    class2_dir = [url2+i for i in os.listdir(url2)]
    len1 = len(class1_dir)
    len2 = len(class2_dir)
    count = len1+len2
    data=np.zeros((count,224,224,3),float)
    labels=np.zeros((count,1),int)
    for i, image_file in enumerate(class1_dir):
        image = loadImage(image_file)
        data[i] = image
        labels[i]=0
        if i%50 == 0: print('Processing Row Data {} of {}'.format(i, count))
    for i, image_file in enumerate(class2_dir):
        image = loadImage(image_file)
        data[len1+i] = image
        labels[len1+i]=1
        if i%50 == 0: print('Processing Row Data {} of {}'.format(i+len1, count))
        print('finished processing data')
    return data , labels

In [ ]:
def built_model():
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',  input_shape = (224, 224, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model


class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

def train(model,train_data,train_labels,nb_epoch=2,batch_size=2,split=0.25,early_stop=True):
    history=LossHistory()
    if early_stop == True:
        early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
        model.fit(train_data, train_labels, batch_size=batch_size, epochs=nb_epoch,
                  validation_split=split, verbose=0, shuffle=True, callbacks=[history, early_stopping])

    else:
        model.fit(train_data, train_labels, batch_size=batch_size, epochs=nb_epoch,
                  validation_split=split, verbose=0, shuffle=True, callbacks=[history])

    return model , history


def predict(model,imgFile):
    img=loadImage(imgFile)
    output = model.predict(img,verbose=0)

    print('')
    return output

def plot_loss(history):
    loss = history.losses
    val_loss = history.val_losses

    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('my model Loss Trend')
    plt.plot(loss, 'blue', label='Training Loss')
    plt.plot(val_loss, 'green', label='Validation Loss')
    plt.xticks(range(0,nb_epoch)[0::2])
    plt.legend()
    plt.show()

In [33]:
dog_data , dog_labels = data_prepared()
cli = built_model()
cli,history=train(cli,dog_data,dog_labels,nb_epoch=2,batch_size=2,split=0.3)
plot_loss(history)

Processed Row Data 0 of 335
Processed Row Data 50 of 335
Processed Row Data 100 of 335
Processed Row Data 150 of 335
Processed Row Data 157 of 335
Processed Row Data 207 of 335
Processed Row Data 257 of 335
Processed Row Data 307 of 335


NameError: name 'plt' is not defined

In [35]:
plot_loss(history)

NameError: name 'plt' is not defined